# Selenium Setup
* Run the code below to prepare the data for the webscraping tool
* This program uses Microsoft Edge
* Note: must download <mark> msedgedriver.exe </mark> from https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
    * Place the file application into a specific folder and put the file location under the variable - <mark> FileLoc </mark>
    
<span style="color: red"> This code WILL take a long time to load. On 11/25/2021, it took approximately 20 hours to run. </span>

**For the user interface**, please use `US-Engineering-Profiles-Sample.csv`, as this code created that spreadsheet.

<span style="color:blue"> **As seen in code, all data comes from asee.org** </span>
    
### Install and Load Code Libraries

In [ ]:
%%capture
pip install -U selenium

In [ ]:
%%capture
pip install msedge-selenium-tools selenium==3.141

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from openpyxl import Workbook

### Navigate DOM to Extract Data

In [2]:
def institutions(driver, t, DF, YRID, DGID, Data_Name, Int_ID, Table_ID):
# Find the Degree Level dropdown box
    DEGREE = driver.find_element_by_id(DGID)
    DEGREE = DEGREE.find_element_by_xpath("..")                          # Go up a level to get the interactable element
    time.sleep(t)                                                        # Wait for data to load
    DEGREE.click()                                                       # Click on the dropdown box
    for each in DEGREE.find_elements_by_xpath("./select/option"):        # Activate all degree levels
        each.click()
        time.sleep(t)
    
# Find the Year dropdown box
    YEAR = driver.find_element_by_id(YRID)
    YEAR = YEAR.find_element_by_xpath("..")                              # Go up a level to get the interactable element
    time.sleep(t)                                                        # Give time for dropdown box to load
    YEAR.click()                                                         # Click on the dropdown box
    for option in YEAR.find_elements_by_xpath("./select/option"):
        option.click()
        time.sleep(t)   
    
# Find the Institution dropdown box
        element = driver.find_element_by_id(Int_ID)
        element = element.find_element_by_xpath("..")                    # Go up a level
        element = element.find_element_by_xpath("./div")                 # Go down to the div interactable element
        time.sleep(t)                                                    # Give time for dropdown box to load
        element.click()                                                  # Click on the dropdown box
        
        i = 0

# Access Data for each Institution
        for opt in element.find_elements_by_css_selector("div.option"):
            i += 1
            #Select Institution
            opt.click()                                                  # Create table for selected school
            time.sleep(t)                                                # Wait for the table to fill

# Scrape Data
            close_table = driver.find_element_by_id(Table_ID)
            table = close_table.find_element_by_xpath("./div/div[3]/div[2]/table[@id]/tbody")    # Find the body of the table with the data
            rows = table.find_elements_by_xpath("./tr")                  # Get all the rows from the table
            for row in rows:  
                data = []
                if len(rows) > 1:                                        # If no data, # of rows = 1
                    columns = row.find_elements_by_xpath("./td")         # Retrieve all values for each row
                    data.append(option.text)                             # Collect data for the year
                    data.append(Data_Name)                               # Record "Degrees Awarded" vs "Enrollment" data
                    for fact in columns:                                 # Add each piece of data to a list 
                        data.append(fact.text)
                    DF.loc[len(DF.index)]=data                           # Add the list to the DataFrame
            element.click()                                              # Reclick dropbox to get next Institution
            if i == 5:
                break
        YEAR.click()                                                     # Reclick dropbox to get next Year

### Setup Variables and DataFrame

In [3]:
ethnicities = ['Non Res Alien M', 'Non Res Alien F',
                'Unknown M', 'Unknown F',
                'Hispanic M', 'Hispanic F',
                'American Indian M', 'American Indian F', 
                'Asian American M', 'Asian American F',
                'African American M', 'African American F',
                'Hawaiian/pac.isl M', 'Hawaiian/pac.isl F',
                'White M', 'White F',
                'Multiracial M', 'Multiracial F']
cols = ['Year', 'Data Type', 'Institution', 
        'Degree Name', 'Discipline Name']+ethnicities+['Total']
DF = pd.DataFrame(columns=cols)

In [4]:
FileLoc = 'C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe'

load_time = 2

DAName = "DegreesAwarded"
DAID = "goButton"
DAYRID = "yrrr"
DADegreeID = "deggg"
DAIntID = "exp"
DATableID = "trans_tbl"

EName = "Enrollment"
EID = "reset_button"
EYRID = "yrrr2"
EDegreeID = "deggg2"
EIntID = "exp2"
ETableID = "trans_tbl2"

BackButtonID = "backsel"

### Run Code

In [ ]:
if __name__=='__main__':
    
    #Create Edge Object
    edgeBrowser=webdriver.Edge(executable_path=FileLoc)
    #Go to Data Source
    edgeBrowser.get("https://shinyapps.asee.org/apps/Profiles/")
    
    #Go to Degrees Awarded WebPage
    edgeBrowser.find_element_by_id(DAID).click()
    institutions(edgeBrowser, load_time, DF, DAYRID, DADegreeID, DAName, DAIntID, DATableID)
    time.sleep(2)
    
    #Go back to Home Page
    edgeBrowser.find_element_by_id(BackButtonID).click()
    time.sleep(2)
    
    #Go to enrollment page
    edgeBrowser.find_element_by_id(EID).click()
    institutions(edgeBrowser, load_time, DF, EYRID, EDegreeID, EName, EIntID, ETableID)
    
    #End Command of Driver
    edgeBrowser.quit()

### Save DataFrame to CSV

In [ ]:
wb = Workbook()
ws = wb.active
wb.save("US-Engineering-Profiles.csv")
DF.to_csv("US-Engineering-Profiles.csv")